In [381]:
import json
import os
import subprocess

import requests


# create a requests Session object 
s = requests.Session()

In [382]:
# get the running server(s) from `jupyter notebook list`
p = subprocess.run(["jupyter", "lab", "list", "--json"], capture_output=True, text=True)
servers = []
for line in p.stdout.splitlines():
    servers.append(json.loads(line))
servers

[{'base_url': '/',
  'hostname': '0.0.0.0',
  'password': False,
  'pid': 7,
  'port': 8888,
  'root_dir': '/root/host_home',
  'secure': False,
  'sock': '',
  'token': '3ff50098f0573f69960120cb2d2ec14b47349445f7d54f24',
  'url': 'http://bf29b2570ad2:8888/',
  'version': '2.14.2'}]

In [383]:
# assume one server for now
# can select by comparing base_url to $JUPYTERHUB_SERVICE_PREFIX
server = servers[0]
jupyter_api = f"{server['url']}api"
token = server['token']
s.headers = {"Authorization": f"token {token}"}

In [384]:
# List sessions using jupyter REST API

sessions = s.get(f"{jupyter_api}/sessions").json()

# sort by activity
# last_activity is ISO8601 strings, sortable without parsing
sessions = sorted(sessions, key=lambda s: s["kernel"]["last_activity"], reverse=True)
sessions

[{'id': '53eb9bc3-46db-41e6-b970-d453e33608db',
  'path': 'Tumor_subcluster-jvsc-0b8f73fb-bd29-48e2-a89d-a376f0fa2cfe9831f10e-2f62-4876-b767-2188b85ebcf8.ipynb',
  'name': 'Tumor_subcluster-0d581faa-5d9a-4de3-bec2-f19662dab7ab.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'e8125a5b-d8c4-4b1d-b432-6dc584ce07b8',
   'name': 'python3',
   'last_activity': '2025-01-18T03:53:05.650681Z',
   'execution_state': 'busy',
   'connections': 2},
  'notebook': {'path': 'Tumor_subcluster-jvsc-0b8f73fb-bd29-48e2-a89d-a376f0fa2cfe9831f10e-2f62-4876-b767-2188b85ebcf8.ipynb',
   'name': 'Tumor_subcluster-0d581faa-5d9a-4de3-bec2-f19662dab7ab.ipynb'}},
 {'id': '2cf72191-9e66-4878-ac73-1d14d2585a62',
  'path': 'Zuani2-jvsc-57015967-640b-4bc6-b8c0-bba26322d3776f6544a3-bbb8-47fe-b183-834b544a21fd.ipynb',
  'name': 'Zuani2-c72492af-6857-4f84-9f05-0033d94c1c73.ipynb',
  'type': 'notebook',
  'kernel': {'id': 'b3095fde-228f-49be-b89f-89e6292c0ef7',
   'name': 'python3',
   'last_activity': '2025-01-18T03:49:

In [385]:
# don't shutdown the most recently active kernel, but shut down all others
if sessions:
    print(f"Not shutting down most recently active session: {sessions[0]['name']}")

for session in sessions[1:]:
    kernel = session['kernel']
    name = session['name']
    if kernel['execution_state'] == 'busy':
        print(f"Not shutting down busy kernel for {name}")
        continue
    if kernel['connections']:
        print(f"Not shutting down kernel with {kernel['connections']} connections: {name}")
        continue
    print(f"Shutting down session {name} idle since {kernel['last_activity']}")
    r = s.delete(f"{jupyter_api}/sessions/{session['id']}")
    r.raise_for_status()

Not shutting down most recently active session: Tumor_subcluster-0d581faa-5d9a-4de3-bec2-f19662dab7ab.ipynb
Not shutting down kernel with 1 connections: Zuani2-c72492af-6857-4f84-9f05-0033d94c1c73.ipynb
